# Agente de Análisis de Costos de Fabricación Aeroespacial

## Descripción

En este ejercicio, crearás un **agente de IA especializado en análisis de datos de fabricación** utilizando el SDK de Agentes de IA de Azure. El agente analizará datos de costos de fabricación de componentes aeroespaciales, calculará métricas estadísticas, identificará tendencias y proporcionará recomendaciones de optimización.

### Casos de Uso
- **Análisis estadístico**: Calcular métricas de costo (promedio, desviación estándar, tendencias)
- **Visualización**: Crear gráficos para identificar componentes más costosos
- **Optimización**: Recomendaciones basadas en datos para mejorar la eficiencia de fabricación

### Herramientas Utilizadas
- **Code Interpreter**: El agente generará dinámicamente código Python para análisis estadístico
- **Azure AI Agents SDK**: Para crear y gestionar el agente

## Prerrequisitos

Asegúrate de que tu archivo `.env` en la raíz del repositorio contiene las variables `PROJECT_ENDPOINT` y `MODEL_DEPLOYMENT_NAME` correctas. Este archivo se reutiliza en todos los laboratorios.

### Cargar librerías y variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

# Import required classes from Azure AI Agents SDK
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole

# Load environment variables from .env file
load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Visualizar los datos a analizar

In [ ]:
script_dir = Path.cwd()  # Use current notebook directory
file_path = script_dir / 'data.txt'

with file_path.open('r') as file:
    data = file.read()
    print("Data to analyze:")
    print(data)

### Conectar al Cliente de Agentes (Agent Client)

Ahora, nos conectaremos al proyecto de Azure AI Foundry usando las credenciales de Azure.

In [ ]:
print("Connecting to Azure AI Agents Client...")
agent_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)
print("Agent client connected successfully")

### Subir el archivo de datos y crear una herramienta de Intérprete de Código (CodeInterpreterTool)

Subimos el archivo de datos al proyecto y creamos una herramienta `CodeInterpreterTool` que pueda acceder a los datos en él.

In [ ]:
print("Uploading file...")
file = agent_client.files.upload_and_poll(
    file_path=str(file_path), purpose=FilePurpose.AGENTS
)
print(f"File uploaded: {file.filename}")

# Create CodeInterpreterTool with uploaded file
code_interpreter = CodeInterpreterTool(file_ids=[file.id])
print("Code interpreter tool created")

### Definir un agente que usa la herramienta CodeInterpreterTool

Creamos un agente de IA con instrucciones específicas sobre su rol y las herramientas que puede utilizar.

In [ ]:
print("Creating agent...")
agent = agent_client.create_agent(
    model=model_deployment,
    name="manufacturing-analyst",
    instructions=(
        """
        Eres un agente de IA especializado en analizar datos de costos de fabricación de componentes aeroespaciales de la empresa.
        Analiza el archivo de datos cargado que contiene costos de fabricación en EUR.
        Usa Python para calcular métricas estadísticas, identificar tendencias de costos y proporcionar
        recomendaciones para la optimización de la fabricación aeroespacial.
        """
    ),
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f"Agent created: {agent.name} (id: {agent.id})")

### Crear un hilo (thread) para la conversación

Iniciamos un hilo donde se ejecutará la sesión de chat con el agente.

In [ ]:
print("Creating thread...")
thread = agent_client.threads.create()
print(f"Thread created (id: {thread.id})")

### Definir la lógica para enviar prompts y procesar respuestas

Esta función encapsula la lógica para enviar un mensaje al agente, procesarlo y obtener la respuesta.

In [ ]:
def send_prompt_to_agent(prompt):
    print(f"Sending prompt: '{prompt}'")
    message = agent_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    print(f"User message created (thread: {thread.id})")

    # Crear y procesar la ejecución (run)
    print("Processing run...")
    run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run completed with status: {run.status}")

    # Comprobar el estado de la ejecución
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    else:
        print("Retrieving agent response...")
        # Mostrar la última respuesta del agente
        last_msg = agent_client.messages.get_last_message_text_by_role(
            thread_id=thread.id,
            role=MessageRole.AGENT,
        )
        if last_msg:
            print("Agent response:")
            print(f"{last_msg.text.value}")

print("Function 'send_prompt_to_agent' defined successfully")

In [ ]:
user_prompt = "¿Cuál es el componente aeroespacial con el costo de fabricación más alto?"

# Send prompt to agent
send_prompt_to_agent(user_prompt)


In [ ]:
user_prompt = "Crea un gráfico de barras basado en texto mostrando el costo de fabricación por componente aeroespacial"

# Send prompt to agent
send_prompt_to_agent(user_prompt)

In [ ]:
user_prompt = "¿Cuál es la desviación estándar de los costos de fabricación? ¿Qué componentes están por encima del promedio?"

# Send prompt to agent
send_prompt_to_agent(user_prompt)

### Obtener el historial de la conversación

Una vez que hayas terminado de interactuar con el agente, puedes ver el registro completo de la conversación.

In [ ]:
print("\nConversation history:\n")
messages = agent_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role}: {last_msg.text.value}\n")

### Limpieza

Finalmente, elimina el agente para liberar recursos.

In [ ]:
print(f"Deleting agent (id: {agent.id})...")
agent_client.delete_agent(agent.id)
print("Agent deleted successfully")